In [ ]:
!pip install kserve

In [2]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1ModelSpec
from kserve import V1beta1ModelFormat

In [3]:
namespace = utils.get_default_target_namespace()

In [ ]:
from model_registry import ModelRegistry

registry = ModelRegistry(server_address="model-registry-service.kubeflow.svc.cluster.local", port=9090, author="mmortari")

lookup_name = "mnist"
lookup_version="v20231206163028"
print("ModelVersion:")
model_version = registry.get_model_version(lookup_name, lookup_version)
print(model_version)
print("ModelArtifact:")
model_artifact = registry.get_model_artifact(lookup_name, lookup_version)
print(model_artifact)
storage_uri = model_artifact.uri
model_format_name = model_artifact.model_format_name
model_format_version = model_artifact.model_format_version
print(f'Will use to compose the KServe isvc: {storage_uri}, {model_format_name}, {model_format_version}')

In [4]:
name='mnist'
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                                 model=V1beta1ModelSpec(
                                     storage_uri=storage_uri,
                                     model_format=V1beta1ModelFormat(name=model_format_name, version=model_format_version),
                                     protocol_version='v2' # see https://kserve.github.io/website/master/modelserving/v1beta1/onnx/#create-the-inferenceservice
                                 )
                               )))

In [ ]:
KServe = KServeClient()
KServe.create(isvc)

In [ ]:
KServe.get(name, namespace=namespace, watch=True, timeout_seconds=120)